## Redes Neurais Profundas com os pacotes Keras e TensorFlow

### Um Classificador de Imagens

In [ ]:
#Importação dos pacotes
import sys
import sklearn

try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Reset no gerador de números randômicos
np.random.seed(42)

## Estudo de caso: construção de um classificador de imagens

In [ ]:
from tensorflow import keras
tf.__version__
keras.__version__

Vamos começar carregando o conjunto de dados MNIST, que 70.000 imagens em tons de cinza de 28 × 28 *pixels* cada, divididas em 10 classes, e ainda dividir os dados em conjuntos de teste e treinamento.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

O conjunto de treinamento possui 60.000 imagens.

In [ ]:
X_train_full.shape

A intensidade de cada *pixel* é representeda como um byte (0 a 255).

In [ ]:
X_train_full.dtype

Vamos dividir o conjunto de treinamento em um conjunto de validação e um conjunto de teste (com 5000 observações). Também dimensionamos as intensidades de pixel para o intervalo de 0 a 1 e convertemos os valores para ponto-flutuante, dividindo por 255.

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

Você pode plotar uma imagem usando a função `imshow()` do pacote `Matplotlib`, por meio de um mapa de cores com a opção `binary`.

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

Os rótulos são representados pelos IDs das classes, de 0 a 9.

In [ ]:
y_train

Os nomes correspondentes são os seguintes:

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

Portanto, a primeira imagem do conjunto de treinamento é um casaco.

In [ ]:
class_names[y_train[0]]

O conjunto de validação contém 5.000 imagens e o conjunto de teste contém 10.000 imagens:

In [ ]:
X_valid.shape

In [ ]:
X_test.shape

Vamos observar uma amostra das imagens no conjunto de dados.

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

### Criação de rede neural profunda baseada em API Sequencial

Agora vamos construir a rede neural profunda! Criaremos um classificador MLP com duas camadas profundas.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

Vamos examinar este código-fonte linha a linha:
 * a primeira linha cria um modelo sequencial; este é o tipo mais simples de modelo Keras para redes neurais profundas, compostas apenas por uma pilha de camadas conectadas sequencialmente. Isso é chamado de **API Sequencial**;
 * em seguida, construímos a primeira camada e a adicionamos ao modelo; é uma camada `Flatten` cuja função é converter cada imagem de entrada em um array 1D; esta camada não possui parâmetros, ela faz apenas um pré-processamento simples; por ser a primeira camada do modelo, você deve especificar `input_shape`, que representa o formato das instâncias (28 x 28 pixels);
 * em seguida, adicionamos a primeira camada oculta com 300 neurônios, usando a função de ativação `relu`;
 * logo após, adicionamos uma segunda camada oculta com 100 neurônios, também usando a função de ativação `relu`;
 * por fim, adicionamos uma camada de saída com 10 neurônios (um por classe), usando a função de ativação `softmax`, porque as classes são exclusivas.

Outra possibilidade de criação da rede neural profunda:

```python
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
```

As camadas da rede neural profunda.

O método `summary()` exibe as informações de todas as camadas do modelo, seus nomes, que são gerados automaticamente (a menos que você especifique ao criar a camada), a forma de saída e o número de parâmetros, que podem ser treináveis ou não treináveis. Neste capítulo estudaremos apenas parâmetros treináveis.

In [ ]:
model.summary()

Observe que as camadas densas geralmente têm **muitos parâmetros**. 

Por exemplo, a primeira camada oculta tem 784 pesos de conexão (número de entradas) × 300 (neurônios da camada), mais 300 pesos de viés (*bias*), que somam 235.500 parâmetros! Isso dá ao modelo bastante flexibilidade para ajustar os dados de treinamento, mas também significa que o modelo corre o risco de *overfitting*, especialmente quando você não tem muitos dados de treinamento.

**Compilando o modelo**

Depois que um modelo é criado, você deve executar o método `compile()` para especificar a função de perda (ou custo, que é usada para minimizar o erro) e o otimizador a ser usado. Opcionalmente, você pode especificar uma lista de métricas extras para calcular durante o treinamento e avaliação.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

Esse código é equivalente a:

```python
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=[keras.metrics.sparse_categorical_accuracy])
```

**Treinamento**

A cada época, `Keras` mostra algumas informações, como a perda e a acurácia, para os conjuntos de teste e treinamento.

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

Épocas do treinamento

In [ ]:
print(history.epoch)

Métricas coletadas em cada época.

In [ ]:
history.history.keys()

Se você usar este dicionário para criar um DataFrame da biblioteca `pandas` e passar como parâmetro ao método `plot()`, você irá visualizar as curvas de aprendizagem, mostradas na Figura a seguir.

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

Você pode perceber que a acurácia do treinamento e a acurácia da validação aumentam continuamente durante o treinamento, enquanto a perda do treinamento e a perda da validação diminuem. Isso é bom! Além disso, as curvas de validação estão próximas das curvas de treinamento, o que significa que não há muito *overfitting*. Nesse caso específico, o modelo parece ter um desempenho melhor no conjunto de validação do que no conjunto de treinamento no início do processo. Mas esse não é o caso: na verdade, o erro de validação é calculado no final de cada época, enquanto o erro de treinamento é calculado usando uma média contínua durante cada época. Portanto, a curva de treinamento deve ser deslocada meio período para a esquerda. Se você fizer isso, verá que as curvas de treinamento e validação se sobrepõem quase perfeitamente no início do treinamento.

### Uso da rede neural para fazer predições

Em seguida, podemos usar o método `predict()` para fazer predisões em novas instâncias. Como não temos novas instâncias reais, usaremos apenas as três primeiras instâncias do conjunto de teste.

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

Como você pode ver, para cada instância o modelo estima uma probabilidade por classe, da classe 0 à classe 9. Se você está preocupado apenas em identificar a classe por meio da probabilidade, independentemente do seu valor, então você pode usar o método `predict_classes()`.

In [ ]:
# Predict probabilities
y_pred = model.predict(X_new)

# For multiclass
y_pred_classes = np.argmax(y_pred, axis=1)

# For binary classification
y_pred_classes = (y_pred > 0.5).astype("int32")

# Display the predicted classes
y_pred_classes


In [32]:
# Predict probabilities
y_pred = model.predict(X_new)

# Convert probabilities to class indices
y_pred_indices = np.argmax(y_pred, axis=1)

# Map indices to class names
predicted_class_names = np.array(class_names)[y_pred_indices]

predicted_class_names


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array(['Ankle boot', 'Pullover', 'Trouser'], dtype='<U11')

É possível ainda visualizar as figuras das classes.

In [ ]:
y_new = y_test[:3]
y_new

In [ ]:
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_new[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()